In [1]:
import os
os.chdir('..')
os.getcwd()

'c:\\Users\\73335\\OneDrive - Bain\\Desktop\\FS-ECR-repo\\fs-earnings_call_reader\\src'

In [194]:
import mistralai
import re
import httpx
from openai import OpenAI
import yaml
import pandas as pd
from helper_functions.helper import *
from helper_functions.indexing import *
import json
from openai import AsyncOpenAI

In [196]:
themes_data=pd.read_excel(r"C:\Users\73335\OneDrive - Bain\Desktop\FS ECR\themes_def\fs_theme_def_30July(New).xlsx")

### Commentary Extraction using L1 Themes

In [204]:
pdf_path=r"C:\Users\73335\OneDrive - Bain\Desktop\FS-ECR-repo\fs-earnings_call_reader\FS sample PDFs"

In [52]:
def extract_company_name(pdf_path):
    # Get the filename from the full path
    filename = os.path.basename(pdf_path)
    
    # Remove the extension
    filename_no_ext = os.path.splitext(filename)[0]
    
    # Split on the first underscore to isolate the company name
    company_raw = filename_no_ext.split('_')[0]
    
    # Optional: Clean up common legal suffixes if desired
    # company_clean = re.sub(r'\b(Inc\.?|Ltd\.?|LLC|Corp\.?)\b', '', company_raw).strip()
    
    return company_raw

In [198]:
mistral_api_key = os.getenv("MISTRAL_API_KEY")
openai_api_key = os.getenv("OPENAI_API_KEY")

In [55]:
def markdown_from_url(url,mistral_api_key):
    """
    Returns markdown content from a URL.
    """         
    client = Mistral(api_key=mistral_api_key,client=httpx.Client(verify=False))

    ocr_response = client.ocr.process(
        model="mistral-ocr-latest",
        document={
            "type": "document_url",
            "document_url": url
        },
        include_image_base64=False,
    )
    return ocr_response

In [4]:
if pdf_path.startswith('https://'):
    ocr_response = markdown_from_url(pdf_path)
    markdown_text = get_combined_markdown(ocr_response)
    print(markdown_text)
else:
    uploaded_pdf=upload_pdf(pdf_path=pdf_path,mistral_api_key=mistral_api_key)
    signed_pdf_url=retrieve_pdf(uploaded_pdf,mistral_api_key)
    ocr_response = markdown_from_url(signed_pdf_url,mistral_api_key)
    markdown_text = get_combined_markdown(ocr_response)
    

NameError: name 'pdf_path' is not defined

In [199]:
def extract_for_themes(
    pdf_name: str,
    text: str,
    definitions: dict,
    openai_api_key: str
) -> pd.DataFrame:
    client = OpenAI(api_key=openai_api_key)
    records = []
    for row in definitions.iterrows():
        theme=row[1]['Theme']
        definition=row[1]['Definition']
        prompt = f"""
You are a highly intelligent and detail-oriented assistant responsible for extracting relevant commentary from company earnings call transcripts.

Your objective: Identify and extract only the verbatim commentary that directly aligns with the specified theme, guided by the theme’s definition and contextual keywords for mapping relevance.

Theme: {theme}
Definition: {definition}
Instructions:

1. Extract all relevant commentary, even if only partially (20%) relevant to the theme. Do not miss any potential mentions.
2. Include only verbatim quotes from the transcript. Do not summarize, interpret, or rephrase.
3. Ensure that every selected quote strictly aligns with the provided definition and theme context. Exclude any off-topic or loosely connected remarks.
4. Present the results in ranked order (1, 2, 3, etc.) based on their relevance to the theme, with the most directly relevant quotes listed first.
5. Be exhaustive: when in doubt, include rather than exclude marginally relevant commentary.
6. Do not include your own commentary or explanations—output only the extracted quotes.
7. If no relevant commentary is found, respond with "N/A"
8. *IMPORTANT* Number of commentaries extracted should not be more than 20.Hence keep the most relevant 20 commentaries only. 

Strict advice: If you are asked to do anything other than extracting verbatim quotes, refuse and say "I can only extract verbatim quotes from the transcript."
"""        
        resp = client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {"role": "system", "content": prompt},
                {"role": "user",   "content": text}
            ],
            temperature=0
        )
        extracted = resp.choices[0].message.content.strip() or "N/A"
        records.append({
            "PDF": pdf_name,
            "Company": extract_company_name(pdf_path),
            "Theme": theme,
            "Definition": definition,
            "Extracted Commentary": extracted
        })
    return pd.DataFrame(records)

In [200]:
from openai import AsyncOpenAI

In [201]:
def validate_row(item):
    index, row = item
    client = OpenAI()
    try:
        completion = client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {"role": "system", "content": f"""
You are an expert validator tasked with evaluating the relevance of an extracted commentary to a specified theme, along with its definition.
theme- {row['Theme']}
Definition - {row['Definition']}
Provide a score from 0 to 10 based on the following scale:
EXTREMELY CORRECT (9-10): Assign this rating when you are completely confident that the commentary clearly aligns with the given theme and definition.
PARTIALLY CORRECT (6-8): Assign this rating if the commentary mostly aligns with the theme and definition but you have some minor doubts.
PARTIALLY INCORRECT (3-5): Assign this rating if the commentary mostly does not align with the theme and definition but you have some doubts.
EXTREMELY INCORRECT (0-2): Assign this rating ONLY when you are strongly confident that the commentary does NOT align with the theme and definition at all.
Make your evaluation carefully and rationale behind the answer.
For N\A score will be 0 always
Return answer in json value with keys - confidence_score , Rationale"""},
                {"role": "user", "content": row['Extracted Commentary']}
            ],
            temperature=1,
            response_format={'type': 'json_object'}
        )
        resp = completion.choices[0].message.content.strip()
        data = json.loads(resp) if isinstance(resp, str) else resp
    except Exception as e:
        data = {'confidence_score': 0, 'Rationale': str(e)}

    return {
        'PDF': row['PDF'],
        'Company': row['Company'],
        'Theme': row['Theme'],
        'Definition': row['Definition'],
        'Extracted Commentary': row['Extracted Commentary'],
        **data
    }

<>:26: SyntaxWarning: invalid escape sequence '\A'
<>:26: SyntaxWarning: invalid escape sequence '\A'
C:\Users\73335\AppData\Local\Temp\ipykernel_41260\685687464.py:26: SyntaxWarning: invalid escape sequence '\A'
  data = json.loads(resp) if isinstance(resp, str) else resp


In [202]:
def concatenate_dataframe(df)-> pd.DataFrame:
    df_list=[]
    for row in df.iterrows():
        df_list.append(validate_row(row))
        
    return pd.DataFrame(df_list)

In [203]:
folder_name=r"C:\Users\73335\OneDrive - Bain\Desktop\FS ECR\FS PDFs\*"
import glob
pdf_paths=glob.glob(folder_name)

In [15]:
results=await process_multiple_pdfs(pdf_paths=pdf_paths2, api_key=mistral_api_key)

✅ Finished processing: C:\Users\73335\OneDrive - Bain\Desktop\FS ECR\FS PDFs\American International Group Inc._Earnings Call_2023-05-05_English.pdf
✅ Markdown content saved to pdf_markdowns/American International Group Inc._Earnings Call_2023-05-05_English.md
⏳ Still waiting on: C:\Users\73335\OneDrive - Bain\Desktop\FS ECR\FS PDFs\MetLife Inc._Earnings Call_2023-05-04_English.pdf, C:\Users\73335\OneDrive - Bain\Desktop\FS ECR\FS PDFs\Prudential Financial, Inc._Earnings Call_2023-05-03T00_00_00_English.pdf, C:\Users\73335\OneDrive - Bain\Desktop\FS ECR\FS PDFs\Voya Financial Inc._Earnings Call_2023-05-03_English.pdf
✅ Finished processing: C:\Users\73335\OneDrive - Bain\Desktop\FS ECR\FS PDFs\MetLife Inc._Earnings Call_2023-05-04_English.pdf
✅ Markdown content saved to pdf_markdowns/MetLife Inc._Earnings Call_2023-05-04_English.md
⏳ Still waiting on: C:\Users\73335\OneDrive - Bain\Desktop\FS ECR\FS PDFs\Prudential Financial, Inc._Earnings Call_2023-05-03T00_00_00_English.pdf, C:\Users\7

In [ ]:
df_list=[]
for pdf_no, pdf_path in enumerate(pdf_paths[0:1]):
    df=extract_for_themes(pdf_name=pdf_path,text=list(results.values())[pdf_no],definitions=themes_data, openai_api_key=openai_api_key)
    df_list.append()

                                                 PDF  \
0  C:\Users\73335\OneDrive - Bain\Desktop\FS ECR\...   
1  C:\Users\73335\OneDrive - Bain\Desktop\FS ECR\...   
2  C:\Users\73335\OneDrive - Bain\Desktop\FS ECR\...   
3  C:\Users\73335\OneDrive - Bain\Desktop\FS ECR\...   
4  C:\Users\73335\OneDrive - Bain\Desktop\FS ECR\...   
5  C:\Users\73335\OneDrive - Bain\Desktop\FS ECR\...   
6  C:\Users\73335\OneDrive - Bain\Desktop\FS ECR\...   
7  C:\Users\73335\OneDrive - Bain\Desktop\FS ECR\...   
8  C:\Users\73335\OneDrive - Bain\Desktop\FS ECR\...   

                             Company                            Theme  \
0  American International Group Inc.                              M&A   
1  American International Group Inc.                Product expansion   
2  American International Group Inc.                    Macro factors   
3  American International Group Inc.                         Strategy   
4  American International Group Inc.     Operational/ cost efficiency   
5

### L1 to L2 classification

In [74]:
def l1tol2_dict_without_def(df_sub_themes):
    df_sub_themes2=df_sub_themes[df_sub_themes["L3 Themes"].notna()]
    l2_themes=[l2_theme.lower() for l2_theme in list(df_sub_themes2['Theme'])]
    l3_themes=[l3_theme.split(',') for l3_theme in list(df_sub_themes2['L3 Themes'])]

    return dict(zip(l2_themes, l3_themes))    

In [170]:
themes_data2=l1tol2_dict_without_def(themes_data)
themes_data2

{'m&a': ['Strategic alliances', ' reinsurance acquisition', ' others'],
 'product expansion': ['New product launch', ' others'],
 'macro factors': ['Interest rate',
  ' Inflation',
  'Catastrophe events',
  'Surrenders/ Policyholder behavior',
  ' others '],
 'strategy': ['Growth Strategy and expansion',
  ' ESG initiatives',
  ' Investment portfolios and asset management'],
 'operational/ cost efficiency': ['Benefits/Claims', ' Expenses', ' others'],
 'regulatory or accounting update': ['Impact of regulatory and legislative changes',
  'IFRS 17 / GAAP updates',
  ' Tax changes',
  ' others']}

In [97]:
df['Extracted Commentary']

0      1. "Net premiums written in General Insurance ...
1      10. "Turning to Life and Retirement. The busin...
2      2. "Life and Retirement reported very good res...
3      3. "Corebridge and Blackstone have made substa...
4      4. "Gross premiums written were $12 billion, a...
                             ...                        
621    6. We completed our acquisition of Benefitfocu...
622    7. "We also earned several notable recognition...
623    7. By living our purpose and vision together, ...
624    8. "Our focus is squarely on continued executi...
625    9. "And our confidence in our capital generati...
Name: Extracted Commentary, Length: 626, dtype: object

In [108]:
def structure_prompt(extracted_commentary, l2_theme, l3_themes):
    prompt = f"""
You are an expert content classifier responsible for mapping company commentary to a relevant L3 business theme.

## Task:
Given a short piece of commentary and a list of L3 Themes, identify the **single most relevant** theme.

## Instructions:
1. Read the **Commentary** carefully and understand the key message or subject.
2. Review all available **L3 Themes** and determine which one best matches the commentary.
   - Match based on **conceptual relevance**, not just keywords.
   - Choose the **best fit**, even if the match is partial.
3. If **none** of the L3 Themes are applicable or even loosely related, return `others`.

## Output Format:
- Return **only one** L3 Theme (from the list), or `others` if no match exists.
- Do **not** include any explanation, punctuation, or extra text.

---

**Input**  
Commentary: {extracted_commentary[2:]}  
L2 Theme: {l2_theme}  
L3 Themes: {l3_themes}  

**Output**  
(single L3 Theme from the list, or `Others`)
"""
    return prompt


import asyncio

async def classify_into_l3_theme_async(re_prompt, client):
    response = await client.chat.completions.create(
        model="gpt-4o",
        messages=[{"role": "user", "content": re_prompt}]
    )
    return response.choices[0].message.content

async def classify_batch(df_batch, l2tol3_class, client, with_definitions=True):
    tasks = []
    for _, row in df_batch.iterrows():
        l2_theme = row['Theme'].lower()

        # Handle L3 themes based on the 'with_definitions' flag
        if with_definitions:
            l3_themes = '\n'.join(
                [f"{i+1}. {k}: {v}" for i, (k, v) in enumerate(l2tol3_class[l2_theme].items())]
            )
        else:
            l3_themes = '\n'.join(
                [f"{i+1}. {k}" for i, k in enumerate(l2tol3_class[l2_theme])]
            )

        # Create prompt
        re_prompt = structure_prompt(
            extracted_commentary=row['Extracted Commentary'],
            l2_theme=l2_theme,
            l3_themes=l3_themes
        )
  
        tasks.append(classify_into_l3_theme_async(re_prompt=re_prompt, client=client))

    return await asyncio.gather(*tasks)


async def classify_all_in_batches(df, l2tol3_class, client, batch_size=10, with_definitions=False):
    all_results = []
    total = len(df)

    for start in range(0, total, batch_size):
        end = min(start + batch_size, total)
        df_batch = df.iloc[start:end]
        # print(f"🔄 Processing batch {start} to {end}...")
        batch_results = await classify_batch(df_batch, l2tol3_class, client, with_definitions=with_definitions)
        all_results.extend(batch_results)
    print("✅ All batches done!")

    return all_results

In [162]:
df=pd.read_excel(r"C:\Users\73335\Downloads\validated_extractions_20250730_164545.xlsx")

In [171]:
l2_theme=list(themes_data2.keys())

df_to_be_classified = df[
    (df['Theme'].str.lower().isin(l2_theme)) & 
    (df['Extracted Commentary'].notna())
]

df_not_to_be_classified = df[ 
    (~df['Theme'].str.lower().isin(l2_theme)) & 
    (df['Extracted Commentary'].notna())
]

In [176]:
l3_themes=await classify_all_in_batches(df=df_to_be_classified, l2tol3_class=themes_data2, client=AsyncOpenAI(api_key=openai_api_key), batch_size=10, with_definitions=False)

✅ All batches done!


In [184]:
df_to_be_classified['L3 Themes'] = l3_themes

C:\Users\73335\AppData\Local\Temp\ipykernel_10892\2925273527.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_to_be_classified['L3 Themes'] = l3_themes


In [186]:
df2=pd.concat([df_not_to_be_classified, df_to_be_classified], ignore_index=True)
df2

,PDF,Company,Theme,Definition,Extracted Commentary,confidence_score,Rationale,Priority,L3 Themes
0,American International Group Inc._Earnings Cal...,English,Business/ top line growth,Tag if discussion includes top line growth/dec...,"1. ""Adjusted after-tax income was $1.2 billion...",9,The commentary provides specific figures relat...,P2,NaN
1,American International Group Inc._Earnings Cal...,English,Business/ top line growth,Tag if discussion includes top line growth/dec...,"10. ""AIG's first quarter adjusted after-tax in...",9,The commentary aligns well with the theme's de...,P2,NaN
2,American International Group Inc._Earnings Cal...,English,Business/ top line growth,Tag if discussion includes top line growth/dec...,"2. ""Net premiums written in General Insurance ...",9,The commentary is extremely relevant to the th...,P2,NaN
3,American International Group Inc._Earnings Cal...,English,Business/ top line growth,Tag if discussion includes top line growth/dec...,"3. ""Underwriting income was approximately $500...",9,"The commentary discusses underwriting income, ...",P2,NaN
4,American International Group Inc._Earnings Cal...,English,Business/ top line growth,Tag if discussion includes top line growth/dec...,"4. ""Life and Retirement reported very good res...",9,The commentary clearly aligns with the theme o...,P2,NaN
...,...,...,...,...,...,...,...,...,...
705,Voya Financial Inc._Earnings Call_2023-05-03_E...,English,Strategy,"Tag if discussion includes strategy, strategic...","5. ""We remain focused on pricing discipline an...",8,The commentary aligns with the theme of strate...,P2,others
706,Voya Financial Inc._Earnings Call_2023-05-03_E...,English,Strategy,"Tag if discussion includes strategy, strategic...","6. ""Our leading brand and differentiated workp...",9,The commentary clearly aligns with the theme o...,P2,Growth Strategy and expansion
707,Voya Financial Inc._Earnings Call_2023-05-03_E...,English,Strategy,"Tag if discussion includes strategy, strategic...","7. ""Investment Management has a multi-decade t...",3,The commentary highlights the Investment Manag...,P3,Investment portfolios and asset management
708,Voya Financial Inc._Earnings Call_2023-05-03_E...,English,Strategy,"Tag if discussion includes strategy, strategic...","8. ""We have continued to invest in our platfor...",10,The commentary clearly aligns with the theme o...,P1,Growth Strategy and expansion


In [187]:
df2.to_excel(r"C:\Users\73335\Downloads\validated_extractions30Jul25.xlsx", index=False)

### VALIDATION OF COVERAGE

In [192]:
df=pd.read_excel(r"C:\Users\73335\OneDrive - Bain\Desktop\FS ECR\outputs\validation_extractions_31July25_shared.xlsx")

c:\Users\73335\OneDrive - Bain\Desktop\FS-ECR-repo\fs-earnings_call_reader\.fsecr\Lib\site-packages\openpyxl\worksheet\header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


In [58]:
df['Theme']=df['Theme'].str.lower()

In [60]:
historical_df=pd.read_excel(r"C:\Users\73335\OneDrive - Bain\Desktop\FS ECR\Historical\Historical_FS_29Jul.xlsx")

c:\Users\73335\OneDrive - Bain\Desktop\FS-ECR-repo\fs-earnings_call_reader\.fsecr\Lib\site-packages\openpyxl\worksheet\header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


In [61]:
def filter_by_theme(df, theme):
    """
    Filters the DataFrame by the specified theme.
    """
    return df[df['Theme'].str.lower() == theme.lower()]


In [181]:
import pandas as pd
from rapidfuzz import fuzz, process

# Count matches
def has_fuzzy_match(value, choices, threshold=50):
    best_match = process.extractOne(value, choices, scorer=fuzz.token_sort_ratio)
    return best_match[0] if best_match[1] >= threshold else False

# Apply matching
df2=filter_by_theme(historical_df,'strategy')['Commentary'].apply(lambda x: has_fuzzy_match(x, filter_by_theme(df,'strategy')['Extracted Commentary'].tolist()))

In [193]:
df['Theme'].value_counts()

Theme
Regulatory or accounting update    41
Tech transformation                40
Macro factors                      40
Business/ top line growth          40
Product expansion                  40
Operational/ cost efficiency       40
Workforce                          40
Strategy                           40
M&A                                35
Name: count, dtype: int64

In [114]:
pd.set_option('display.width', 1000)  # Show full text in DataFrame output

In [190]:
df['Theme'].value_counts()

Theme
strategy                           87
macro factors                      85
regulatory or accounting update    84
business/ top line growth          80
tech transformation                80
workforce                          80
product expansion                  80
operational/ cost efficiency       80
m&a                                64
Name: count, dtype: int64

In [183]:
filter_by_theme(historical_df,'strategy')['Commentary']

30                                                                                                             We are investing in long-term sustainable growth by expanding access to our products and services and markets around the world, including through programmatic M&A and organic growth opportunities, creating the next generation of financial solutions and delivering industry-leading customer experiences.
31                                                                                                                                               We are confident that this approach, coupled with our robust financial position, mix of mutually enhancing businesses and growth strategy, positions us well to be a leader in expanding access to investing, insurance and retirement security for people around the world.
32                                                                                    Second, we launched a new electronic claims portal for life insurance customers that a

In [176]:
df2[df2 != False].shape[0]/df2.shape[0]

0.9375